# Project 6- Pipeline 1 - Download

This is a mostly working version of my download pipeline.  I prefer to use the commandline tool, Download.py

The documentation for this tool is non-existant, except for these notes.

Download.py accepts 3 types of arguments on the commandline.  First, it will check if any arguments are files, and attempt to read those files like yaml files containing a dictionary.  Second, it will look for a arugument like "location=...".  This arugment will be used to change the location of the database used to store the downloaded information.  The locations are defined in a file called 'credentials.yml'.  Third, any strings that don't match either of other special types will be used as a categories to be searched.

In [1]:
import yaml
import os
os.chdir('.') # Stay in the directory. Other people need to go up a level to make this work.

import lib.database_module as db
import lib.wiki_module_bak as wiki
import lib.wiki_module as wiki2
import lib.encoding_funcs as enc

from IPython.display import IFrame


### BONUS: Present: Command is executed via command line script	

see Download.py in lib (or maybe /bin depending how much refactoring I find the time to do.)

### Acquire: Get categories to query from a yaml file

In [2]:
with open('data/categories.yml', 'r') as f:
    category =  yaml.load(f) 

### Acquire: Query Wikipedia by category for pages				
Bonus: write your own method

In [3]:
category.keys()[0]

'category'

In [4]:
pages_list_tupled = []
for cate in category[category.keys()[0]]:
    pages_list_tupled.append(wiki.query_category(cate))
    pages_list_tupled

ConnectionError: HTTPSConnectionPool(host='en.wikipedia.org', port=443): Max retries exceeded with url: /w/api.php?action=query&list=categorymembers&cmtitle=Category:sports%20cars&cmlimit=500&titles=sports%20cars&format=json (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x10d906890>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

    for cate2 in pages_list_tupled:
        db.create_or_update_category_in_database(cate[0],cate[1],'local')
        for page in cate2[2]:
        temp = wiki.query_page(page[0])
        
#### This fails so hard, it needs to be in markdown so it can serve as warning sign.


## My Version of query and :

### Acquire: Write category info to database
### Acquire: Query Wikipedia by page id for content	
Bonus: write your own method
### Acquire: Write page content to database

In [ ]:
import lib.download_mine as download
import sys
import yaml
import os 
import re
import lib.database_module as db

def fetch_and_store_page(title, page_id,category_number,db_location):
    if re.search('Category:',title):
            pass
    text = get_text(title)
    db.create_or_update_page_in_database(page_id,category_number,title,text,db_location)

def get_text(title):
    wiki_page = wikipedia_get_page(title)
    soup = BeautifulSoup(wiki_page[wiki_page.keys()[0]]['extract'], "html.parser")
    return soup.get_text()

  

In [ ]:
category_list = get_categories()
#for cat in category_list:
for cat in category_list.items():
    category = down.wikipedia_get_page(cat, category=True)
    cate_id = cat
    cate_name = category[cat]['title'].split(':')[1]
    #print cate_id, cate_name
    pages = wikipedia_get_pages_for_category(cate_name)
    for page in pages:
        db.fetch_and_store_page(page['title'],page['pageid'],cate_id,db_location)


In [ ]:
for cate2 in pages_list_tupled:
    #db.create_or_update_category_in_database(cate[0],cate[1],'local')
    for page in cate2[2]:
        temp = wiki2.query_page(str(page[0]))
        print temp
        




### BONUS: 
### Identify: Provide a detailed analysis of the database schema in your report	

In [5]:
IFrame("doc/database_design.pdf",width = 600, height=500)

The Database went through many iterations.  The database team was composed of 3 people, and had at least 2 contradictory points of view at all times.  Suketu Kothari consistently argued for consistency.  My point of view was for to relax rigor in favor of speed.  I planned for the integrity of the database to be maintained programatically and strucutrally.  Suketu preferred to handle all consistency with the structure of the database.  The final desgin is a fair compromise.  

The main tables are the Category table and the page table.  Because of the nature of the data source, a page could belong to multiple categories.  In fact, this is the case for categories we have queries.  To support a many to many relationship, there is a junction table called "page-category".  

The last 2 tables support the vectors for the search function.  These tables link back to the page and caegory tables, respectively.  The other field is an array of double precision numbers or floats.  Filling these tables has been delegated to the encoding team.

The database module and function calls have been developed in an ad hoc manner.  The functions are developed to fulfill an immedate need and will need to be rewritten to reduce redundant code and expand functionality.

### Further discussion: 

I've chosen to use my on Download function, from the command line.  There are few difference between my process and process from the wiki team.

1. The wiki team didn't handle unicode well, at least in the first versions.  I tried to build a system that would tolerate unicode characters.  If there is a problem encoding those characters for the vectorization, I would choose to extract words with unicode characters, strip the unicode, and append the stripped words to the text before vectorization.

1. Inclusion of the summary as a return doesn't make sense to store in the database.  Putting redundant information in the database violates my database design principles.  When needed, the snippet can be recovered from the stored text of the page.

1. The wiki team's module doesn't handle categories with more than 500 pages.  The api has a limited return of 500 pages per request.  When the call exceeds 500 pages, an additional key/value pair is returned with information to continue the query.  To call all pages for large categories, the "pages for a category" query needs to be recurrsive in cases of categories with many pages.